In [ ]:
import numpy as np
from lib.helpers import heaviside_step


class Biomassa:
    def __init__(
        self,
        t0,
        t,
        wn,
        w0,
        alpha,
        n0: int,
        sr: float,
        m: float,
        ph: list,
        doc: list,
        f_uia,
        f_o2,
        f_temp,
        score_csc,
        feeding_rate,
        alpha2,
        alpha3,
        alpha4,
        alpha5,
        alpha6,
        final_doc: int = 120,
    ):
        self.t0 = t0
        self.t = t
        self.wn = wn
        self.w0 = w0
        self.alpha = alpha
        self.n0 = n0
        self.sr = sr
        self.m = m
        self.ph = ph
        self.doc = doc
        self.final_doc = final_doc

        self.f_uia = f_uia
        self.f_o2 = f_o2
        self.f_temp = f_temp
        self.score_csc = score_csc
        self.feeding_rate = feeding_rate

        self.alpha2 = alpha2
        self.alpha3 = alpha3
        self.alpha4 = alpha4
        self.alpha5 = alpha5
        self.alpha6 = alpha6


    def wt(self):
        fr = (
            self.alpha * (self.t - self.t0) 
            + self.alpha2 * self.f_temp(self.t) 
            + self.alpha3 * self.f_o2(self.t) 
            + self.alpha4 * self.f_uia(self.t)
            + self.alpha5 * self.score_csc 
            + self.alpha6 * self.feeding_rate
        )

        return (
            self.wn ** (1 / 3)
            - (self.wn ** (1 / 3) - self.w0 ** (1 / 3))
            * np.exp(-1 * fr)
        ) ** 3

    def population(self):
        t = self.t
        ph = self.ph
        partial_harvest = [
            ph[i] * heaviside_step(t - j) for i, j in enumerate(self.doc)
        ]

        if t >= self.final_doc:
            partial_harvest.append(
                (self.sr - sum(ph)) * heaviside_step(t - self.final_doc)
            )

        result = self.n0 * (np.exp(-self.m * t) - sum(partial_harvest))
        return result

    def biomassa(self):
        # biomassa in gram
        return self.wt() * self.population()

    def biomassa_constant(self):
        return self.n0 * self.wt()


In [ ]:
from lib.helpers import source_data, score_csc_compute
from lib.uem.feeding_rate import feeding_rate

f_uia, f_o2, f_temp, temperature = source_data(
    # path = "data/growth_full2.csv",
    path = None,
    temp_suitable_min = 25,
    temp_suitable_max = 33,
    temp_optimal_min = 28,
    temp_optimal_max = 32,
    do_suitable_min = 4,
    do_suitable_max = 10,
    do_optimal_min = 6,
    do_optimal_max = 9,
    ua_suitable_min = 0.00,
    ua_suitable_max = 0.16,
    ua_optimal_min = 0.00,
    ua_optimal_max = 0.06,
)

csc_suitable_min = 0.00
csc_suitable_max = 5
csc_optimal_min = 0.00
csc_optimal_max = 3

In [ ]:
import functools

@functools.lru_cache(maxsize=18000)
def single_wt1(t, sr, m, alpha, alpha2, alpha3, alpha4, alpha5,  alpha6):
    try:
        feedRate = feeding_rate(0, float(temperature[temperature["DOC"]==t]["Temp"]), 0)
    except  :
        feedRate = 0

    if t == 0:
        score_csc = score_csc_compute(0/1000, 1000, csc_suitable_min, csc_suitable_max, csc_optimal_max)
    else:
        _, bio_1 = single_wt1(t-1, sr, m, alpha, alpha2, alpha3, alpha4, alpha5, alpha6)
        score_csc = score_csc_compute(bio_1/1000, 1000, csc_suitable_min, csc_suitable_max, csc_optimal_max)
        
    obj = Biomassa(0, t, 40, 0.05, alpha, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, feedRate, score_csc, 
        alpha2, alpha3, alpha4, alpha5, alpha6, final_doc=120)

    return  obj.wt(), obj.biomassa()


def multi_wt(T, alpha, alpha2, alpha3, alpha4, alpha5, alpha6):
    sr = 0.92
    m = -np.log(sr)/T[-1]
    res = np.asarray([single_wt1(t, sr, m, alpha, alpha2, alpha3, alpha4, alpha5, alpha6)[0] for t in T])
    return res

In [ ]:
from scipy.optimize import curve_fit
curve_fit(multi_wt, [1, 2, 3, 4, 5, 6, 7], [0.0145, 0.028, 0.0415, 0.055, 0.0685, 0.082, 0.0955], p0=[0.05, 0.05, 0.05,0.05, 0.05,0.05] , absolute_sigma=True,  method="dogbox", ftol=1e-05)

In [3]:
import pandas as pd

df = pd.read_csv("data/growth_full.csv", sep=",")

In [ ]:
curve_fit(multi_wt, df["DOC"].tolist(), df["ABW"].tolist(), p0=[0.05, 0.05, 0.05,0.05, 0.05,0.05] , method="dogbox", ftol=1e-05)

# Plot

In [ ]:
T = 120
sr = 0.92
m = -np.log(sr)/T

weight = []
bio = []
index = list(range(T+1))
for t in index:
    wt, biomass = single_wt1(t, sr, m,  0.01604904,  0.09431816,  0.00844429,  0.02735245,  0.05, -0.25474762)
    weight.append(wt)
    bio.append(biomass)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(index[:119], bio[:119])

In [ ]:
plt.plot(index[:119], weight[:119])

In [ ]:
obj = Biomassa(0, 1, 40, 0.05,  0.01743061, 100, sr, m, [0.1, 0.1, 0.1], [60, 70, 80], f_uia, f_o2, f_temp, 0, 0, 
        -0.14014172, -0.0880867 , -0.0330906,  0.05, -0.04210808, final_doc=120)

In [ ]:
obj.wt()

In [ ]:
np.exp(-1 * ( 0.01743061 * (0) +  -0.14014172 * f_temp(0) 
                            + -0.0880867 * f_o2(0) + -0.0330906 * f_uia(0)))
                            # + 1 * score_csc + -0.04210808 * feeding_rate))

In [ ]:
(40**(1/3) - (40**(1/3) - 0.05**(1/3))*1.3)**3

## LMFIT

In [ ]:
from lmfit import Model

In [ ]:
xa = np.array([0.5, 0.53, 0.56, 0.59, 0.62, 0.65, 0.68, 0.7, 0.72, 0.74,
               0.76, 0.78, 0.8, 0.82])
ya = np.array([0.40168, 0.40103999999999995, 0.40027999999999997, 0.39936,
               0.39828, 0.397, 0.39544, 0.39424000000000003, 0.39292,
               0.39144, 0.38976, 0.38788, 0.38580000000000003, 0.38348])

def modelfunc(x, a, b, c):
    return (1 + c)/(1 + np.exp((a-x)/b))

my_model = Model(modelfunc)
params = my_model.make_params(a=1, b=-0.1, c=-0.5)
result = my_model.fit(ya, params, x=xa)

In [ ]:
wt_model = Model(multi_wt)
params = wt_model.make_params(alpha=0.05, alpha2=0.05, alpha3=0.05, alpha4=0.05, alpha5=0.05, alpha6=0.05)
res = wt_model.fit(df["ABW"].tolist(), params, T=df["DOC"].tolist())

In [ ]:
f_o2(102)

In [1]:
from lib.parameter_estimation import Estimate

In [4]:
obj = Estimate(0, 0.05, 40, 1000, 0.92, [0.1, 0.1, 0.1], [60, 80, 100],  df["DOC"].tolist(), df["ABW"].tolist(), 120)

In [5]:
obj.fitting()

(0.016049035645296066,
 0.09431816488330128,
 0.008444285811812334,
 0.02735245164011519,
 0.05,
 -0.2547476247186072)